In [ ]:
from rcnn_utils import get_images_target, get_object_detection_model, decode_prediction
import torch
import numpy as np
from tqdm import tqdm
from torchvision import transforms
from torchmetrics.detection.mean_ap import MeanAveragePrecision

In [ ]:
# Connect to the GPU if one exists.
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("Using: ", device)
torch.cuda.empty_cache()

In [ ]:
# Takes 17 seconds

# Get data stores in seal counter directory
data_path = "../../../seal_detector/Data"

# Load Data
train_img_data = np.load(f"{data_path}/train_images.npy", allow_pickle=True)
train_bb_data = np.load(f"{data_path}/train_bb_data.npy", allow_pickle=True)

val_img_data  = np.load(f"{data_path}/val_images.npy", allow_pickle=True)
val_bb_data = np.load(f"{data_path}/val_bb_data.npy", allow_pickle=True)

test_img_data  = np.load(f"{data_path}/test_images.npy", allow_pickle=True)
test_bb_data = np.load(f"{data_path}/test_bb_data.npy", allow_pickle=True)

In [ ]:
model_path = "../..//Models/rcnn_resnet_v1_unfrozen_transformations_step_50_with_backbone_weights_50"

# Load model
model = get_object_detection_model(1)
model.load_state_dict(torch.load(model_path))

# Putting model on GPU
_ = (
    model
    .to(device)
    .eval()
    )

In [ ]:
# Extract Sub-images and bounding box data for training data
training_sub_images, training_target = get_images_target(train_img_data, train_bb_data)

In [ ]:
val_sub_images, val_target = get_images_target(val_img_data, val_bb_data)

In [ ]:
test_sub_images, test_target = get_images_target(test_img_data, test_bb_data)

In [ ]:
def get_prediction_count(pred):
    return len(pred["boxes"])

In [ ]:
def calculate_map(sub_images, targets, model, score_threshold=.9):
    trans = transforms.Compose([transforms.ToTensor()])
    metric = MeanAveragePrecision(extended_summary=True)
    predictions = []
    predicted_counts = []
    actual_count = []

    for i in tqdm(range(len(sub_images))):
        image = sub_images[i]
        
        # Transform image to be passed to the model
        image = (
            trans(image)
            .unsqueeze(0)
            .to(device)
            )
        
        # Make prediction on sub_image
        raw_prediction = model(image)[0]
        boxes, labels, scores= decode_prediction(raw_prediction, score_threshold)
        predictions.append(
            {
                "boxes": boxes, 
                "labels":labels, 
                "scores":scores
            }
        )

        # Record predicted and actual counts
        predicted_counts.append(len(boxes))
        actual_count.append(len(targets[i]["boxes"]))
    
    # Numerical Count Difference
    predicted_counts = np.array(predicted_counts)
    actual_count = np.array(actual_count)
    count_dif = abs(predicted_counts - actual_count)
    over_counts = (predicted_counts > actual_count)
    under_counts = (predicted_counts < actual_count)

    # Print count metrics
    print(f"Totals Seals: {actual_count.sum()} Total Seals Predicted: {predicted_counts.sum()} Total Error: {count_dif.sum()}")
    print(f"Average Error per sub-image: {count_dif.mean()}")
    print(f"Average Percent Error per sub-iomage: {(count_dif / actual_count).mean()}")
    print(f"Sub-images Overcounted: {over_counts.sum()}, Average Overcount Diff:{count_dif[over_counts].mean()}")
    print(f"Sub-images Undercounted: {under_counts.sum()}, Average Overcount Diff:{count_dif[under_counts].mean()}")
    
    # Calculate MAP
    metric.update(predictions, targets)
    result = metric.compute()
    return result

In [ ]:
threshold = .65

In [ ]:
calculate_map(training_sub_images, training_target, model, threshold)

In [ ]:
calculate_map(val_sub_images, val_target, model, threshold)

In [ ]:
calculate_map(test_sub_images, test_target, model, threshold)